# Model Inference

> Fill in a module description hered

In [ ]:
# | default_exp model

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from typing import Optional, List

import torch
from torch import nn
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer
from torchtyping import TensorType

from toolformer.api import BaseAPI

/Users/education/DATA/projects/ai/toolformer/env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# detect
# wait for end of token
# extract
# excute
# add the result to the input
# continue

In [ ]:
#| export
class ToolFormer(nn.Module):
    def __init__(
        self,
        model: AutoModelForCausalLM,
        apis: List[BaseAPI],
        config: dict
    ):
        super().__init__()
        self.model = model
        self.apis = apis
        self.config = config
        self.is_calling_api: bool = False
        
        # TODO: make a config class contains token_id
        tokenizer = AutoTokenizer.from_pretrained(self.config["tokenizer"]["path"])
        
        start_character = config["data_generator"]["api_start_character"]
        end_character = config["data_generator"]["api_end_character"]
        output_character = config["data_generator"]["api_output_character"]
        
        self.api_start_token_id = tokenizer(f' {start_character}', return_tensors="pt")["input_ids"][0]
        self.api_end_token_id = tokenizer(end_character, return_tensors="pt")["input_ids"][0]
        self.api_output_token_id = tokenizer(f'{output_character}', return_tensors="pt")["input_ids"][0]
    
    def sampling(self, probs: TensorType["batch_size", "seq_len"]) -> TensorType[1]:
        return torch.argmax(probs, dim=-1)
    
    def forward(
        self,
        input_ids: TensorType["batch_size", "seq_len"],
        attention_mask: Optional[TensorType["batch_size", "seq_len"]]=None,
        max_new_tokens: int = 10,
        **kwargs
    ) -> TensorType["batch_size", "seq_len"]:
                
        generated_ids = input_ids
        for _ in range(max_new_tokens):
            output = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                **kwargs
            )
            
            logits = output.logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            _, top_k_idx = torch.topk(probs, k=5, dim=-1)
            
            if self.is_calling_api is True:
                if self.api_end_token_id in top_k_idx:
                    self.is_calling_api = False
                    pred_idx = self.api_end_token_id
                else:
                    pred_idx = self.sampling(probs)
            else:
                if self.api_start_token_id in top_k_idx:
                    self.is_calling_api = True
                    pred_idx = self.api_start_token_id
                else:
                    pred_idx = self.sampling(probs)
            
            generated_ids = torch.cat([generated_ids, pred_idx.unsqueeze(0)], dim=-1)
        
        return generated_ids